<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/Fetching_Data_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pandas


In [ ]:
import requests
import pandas as pd
import time
from IPython.display import clear_output


In [ ]:
# Fetch user inputs
line_id = input("Enter the line id: ")
stop_point_id = input("Enter the stop point id: ")
direction = input("Enter the direction (inbound/outbound): ")

Enter the line id: 22
Enter the stop point id: 490000173RF
Enter the direction (inbound/outbound): inbound


In [ ]:
def fetch_arrival_predictions(line_id, stop_point_id, direction=None, destination_station_id=None):
    base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"

    params = {
        'naptanId': stop_point_id,
    }
    if direction:
        params['direction'] = direction
    if destination_station_id:
        params['destinationStationId'] = destination_station_id

    response = requests.get(base_url, params=params)
    data = response.json()

    # Parse the data
    predictions = []
    for item in data:
        predictions.append({
            'Line': item['lineName'],
            'Destination': item['destinationName'],
            'Expected Arrival': item['expectedArrival'],
            'Platform': item.get('platformName', 'N/A'),
            'Current Location': item.get('currentLocation', 'N/A'),
        })

    # Display the data in a readable form using pandas
    df = pd.DataFrame(predictions)
    return df

# Refresh the data every 15 seconds
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)
    clear_output(wait=True)
    print(df)
    time.sleep(15)  # Refresh every 15 seconds

In [ ]:
def fetch_arrival_predictions(line_id, stop_point_id, direction=None, destination_station_id=None):
    base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"

    params = {
        'naptanId': stop_point_id,
    }
    if direction:
        params['direction'] = direction
    if destination_station_id:
        params['destinationStationId'] = destination_station_id

    response = requests.get(base_url, params=params)
    data = response.json()

    # Parse the data
    predictions = []
    for item in data:
        predictions.append({
            'Line ID': item['lineId'],
            'Vehicle ID': item['vehicleId'],
            'Stop Point ID': item['naptanId'],
            'Station Name': item['stationName'],
            'Direction': item['direction'],
            'Expected Arrival': item['expectedArrival'],
        })

    # Display the data in a readable form using pandas
    df = pd.DataFrame(predictions)
    return df
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)
    clear_output(wait=True)
    print(df)
    time.sleep(15)

Streaming output truncated to the last 5000 lines.
182  2024-06-20T14:08:10Z  
183  2024-06-20T14:26:29Z  
184  2024-06-20T14:29:03Z  

[185 rows x 6 columns]
    Line ID Vehicle ID Stop Point ID                Station Name Direction  \
0        22    BV66VHY   490014150KA              Walpole Street   inbound   
1        22    YY66OZG    490014186I      Wandsworth Bridge Road  outbound   
2        22    BV66VHY    490006224G         Duke of York Square   inbound   
3        22    LJ62KHV    490000206H       Sloane Square Station   inbound   
4        22    BV66ZRZ    490005138N               Wilton Street  outbound   
..      ...        ...           ...                         ...       ...   
180      22    BV66VJU   490000093PA          Green Park Station  outbound   
181      22    BV66VHY    490000119P    Hyde Park Corner Station   inbound   
182      22    YY66OYW    490011281N               Putney Common  outbound   
183      22    YY66OZG    490005076E  Hobury Street / Worlds 

21/06/2024



In [2]:
!pip install pandas requests pytz

import pandas as pd
import requests
import time
import pytz
from datetime import datetime
from IPython.display import clear_output

# TfL API details
api_url = 'https://api.tfl.gov.uk/StopPoint/490008660N/Arrivals'
headers = {'accept': 'application/json'}

# Define time zones
gmt = pytz.timezone('GMT')
bst = pytz.timezone('Europe/London')

def fetch_bus_data():
    response = requests.get(api_url, headers=headers)
    data = response.json()
    df = pd.json_normalize(data)

    # Convert times from GMT to BST
    df['expectedArrival'] = pd.to_datetime(df['expectedArrival']).dt.tz_convert(bst)
    df['scheduledArrival'] = pd.to_datetime(df['scheduledArrival']).dt.tz_convert(bst)

    # Adjust to local time (laptop time)
    df['expectedArrival'] = df['expectedArrival'].dt.tz_localize(None)
    df['scheduledArrival'] = df['scheduledArrival'].dt.tz_localize(None)

    return df

def calculate_ewt(df):
    df['ewt'] = (df['expectedArrival'] - df['scheduledArrival']).dt.total_seconds() / 60.0
    return df

while True:
    # Clear the previous output
    clear_output(wait=True)

    # Fetch and process data
    df = fetch_bus_data()
    df = calculate_ewt(df)

    # Display the latest data
    print(df[['lineId', 'vehicleId', 'expectedArrival', 'scheduledArrival', 'ewt']])

    # Wait for 30 seconds before fetching data again
    time.sleep(30)


KeyError: 'scheduledArrival'

In [3]:
!pip install pandas requests pytz

import pandas as pd
import requests
import time
import pytz
from datetime import datetime
from IPython.display import clear_output

# TfL API details
api_url = 'https://api.tfl.gov.uk/StopPoint/490008660N/Arrivals'
headers = {'accept': 'application/json'}

# Define time zones
gmt = pytz.timezone('GMT')
bst = pytz.timezone('Europe/London')

def fetch_bus_data():
    response = requests.get(api_url, headers=headers)
    data = response.json()
    df = pd.json_normalize(data)

    # Check if 'expectedArrival' and 'scheduledArrival' columns are present
    if 'expectedArrival' in df.columns and 'scheduledArrival' in df.columns:
        # Convert times from GMT to BST
        df['expectedArrival'] = pd.to_datetime(df['expectedArrival']).dt.tz_convert(bst)
        df['scheduledArrival'] = pd.to_datetime(df['scheduledArrival']).dt.tz_convert(bst)

        # Adjust to local time (laptop time)
        df['expectedArrival'] = df['expectedArrival'].dt.tz_localize(None)
        df['scheduledArrival'] = df['scheduledArrival'].dt.tz_localize(None)

        return df
    else:
        print("Expected columns are not present in the data.")
        return pd.DataFrame()  # Return an empty DataFrame if the columns are missing

def calculate_ewt(df):
    if not df.empty:
        df['ewt'] = (df['expectedArrival'] - df['scheduledArrival']).dt.total_seconds() / 60.0
    return df

while True:
    # Clear the previous output
    clear_output(wait=True)

    # Fetch and process data
    df = fetch_bus_data()
    if not df.empty:
        df = calculate_ewt(df)
        # Display the latest data
        print(df[['lineId', 'vehicleId', 'expectedArrival', 'scheduledArrival', 'ewt']])
    else:
        print("No data to display.")

    # Wait for 30 seconds before fetching data again
    time.sleep(30)



Expected columns are not present in the data.
No data to display.


KeyboardInterrupt: 

In [5]:
!pip install requests pandas
import requests
import pandas as pd
import time
from IPython.display import clear_output
# Fetch user inputs
line_id = input("Enter the line id: ")
stop_point_id = input("Enter the stop point id (naptanId): ")
direction = input("Enter the direction (inbound/outbound): ")


Enter the line id: 214
Enter the stop point id (naptanId): 490008660N
Enter the direction (inbound/outbound): outbound


In [6]:
!pip install requests pandas

import requests
import pandas as pd
import time
from IPython.display import clear_output

# Fetch user inputs
line_id = input("Enter the line id: ")
stop_point_id = input("Enter the stop point id (naptanId): ")
direction = input("Enter the direction (inbound/outbound): ")

def fetch_data(line_id, stop_point_id):
    line_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"
    stop_url = f"https://api.tfl.gov.uk/StopPoint/{stop_point_id}/Arrivals"

    line_response = requests.get(line_url).json()
    stop_response = requests.get(stop_url).json()

    line_df = pd.DataFrame(line_response)
    stop_df = pd.DataFrame(stop_response)

    return line_df, stop_df

while True:
    clear_output(wait=True)

    line_df, stop_df = fetch_data(line_id, stop_point_id)

    print(f"Arrivals based on line id ({line_id}):")
    display(line_df[['vehicleId', 'destinationName', 'expectedArrival', 'timeToStation']])

    print(f"Arrivals based on stop point id ({stop_point_id}):")
    display(stop_df[['lineName', 'destinationName', 'expectedArrival', 'timeToStation']])

    time.sleep(30)


Arrivals based on line id (214):


,vehicleId,destinationName,expectedArrival,timeToStation
0,LA19JZW,Highgate Village,2024-06-21T10:38:05Z,423
1,LA19KBF,Moorgate Station,2024-06-21T10:42:33Z,691
2,LA19KAU,Highgate Village,2024-06-21T10:32:55Z,113
3,LA19KCE,Highgate Village,2024-06-21T10:39:40Z,518
4,LA19KAO,Moorgate Station,2024-06-21T10:59:41Z,1719
...,...,...,...,...
262,LA19KBO,Moorgate Station,2024-06-21T10:39:18Z,496
263,LA19KBF,Moorgate Station,2024-06-21T10:41:42Z,640
264,LA19KBF,Moorgate Station,2024-06-21T10:59:13Z,1691
265,LA19KBV,Moorgate Station,2024-06-21T10:31:20Z,18


Arrivals based on stop point id (490008660N):


,lineName,destinationName,expectedArrival,timeToStation
0,214,Highgate Village,2024-06-21T10:51:32Z,1230
1,214,Highgate Village,2024-06-21T10:33:10Z,128
2,214,Highgate Village,2024-06-21T11:00:15Z,1753
3,214,Highgate Village,2024-06-21T10:45:13Z,851
4,88,Parliament Hill Fields,2024-06-21T10:50:36Z,1174
5,88,Parliament Hill Fields,2024-06-21T10:31:57Z,55
6,88,Parliament Hill Fields,2024-06-21T10:58:52Z,1670


KeyboardInterrupt: 

In [7]:
!pip install requests pandas

import requests
import pandas as pd
import time
from IPython.display import clear_output

# Fetch user inputs
line_id = input("Enter the line id: ")
stop_point_id = input("Enter the stop point id (naptanId): ")
direction = input("Enter the direction (inbound/outbound): ")

def fetch_data(line_id, stop_point_id):
    line_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"
    stop_url = f"https://api.tfl.gov.uk/StopPoint/{stop_point_id}/Arrivals"

    try:
        line_response = requests.get(line_url)
        stop_response = requests.get(stop_url)

        line_response.raise_for_status()
        stop_response.raise_for_status()

        line_data = line_response.json()
        stop_data = stop_response.json()

        line_df = pd.DataFrame(line_data)
        stop_df = pd.DataFrame(stop_data)

        return line_df, stop_df, None
    except requests.exceptions.RequestException as e:
        return None, None, str(e)

while True:
    clear_output(wait=True)

    line_df, stop_df, error = fetch_data(line_id, stop_point_id)

    if error:
        print("API not responding:", error)
    else:
        print(f"Arrivals based on line id ({line_id}):")
        display(line_df)

        print(f"Arrivals based on stop point id ({stop_point_id}):")
        display(stop_df)

    time.sleep(30)


Arrivals based on line id (214):


,$type,id,operationType,vehicleId,naptanId,stationName,lineId,lineName,platformName,direction,...,destinationNaptanId,destinationName,timestamp,timeToStation,currentLocation,towards,expectedArrival,timeToLive,modeName,timing
0,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",1042836920,1,LA19KBK,490000036R,Camden Town Station / Camden Street,214,214,R,outbound,...,,Moorgate Station,2024-06-21T10:42:18.3781289Z,1473,,"Kings Cross, Euston Or Oxford Street",2024-06-21T11:06:51Z,2024-06-21T11:07:21Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
1,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-768714504,1,LA19KBO,490011756E,Royal College Street,214,214,U,outbound,...,,Moorgate Station,2024-06-21T10:42:18.3781289Z,95,,Kings Cross,2024-06-21T10:43:53Z,2024-06-21T10:44:23Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
2,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",102935163,1,LA19KAJ,490000169ZA,Old Street Station,214,214,N,inbound,...,,Highgate Village,2024-06-21T10:42:18.3781289Z,1641,,Hoxton Or Angel Islington,2024-06-21T11:09:39Z,2024-06-21T11:10:09Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
3,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-2134033844,1,LA19KAO,490000036R,Camden Town Station / Camden Street,214,214,R,outbound,...,,Moorgate Station,2024-06-21T10:42:18.3781289Z,1043,,"Kings Cross, Euston Or Oxford Street",2024-06-21T10:59:41Z,2024-06-21T11:00:11Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
4,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-1711415183,1,LA19KBK,490011199S,Kentish Town Road / Prince of Wales Road,214,214,KD,outbound,...,,Moorgate Station,2024-06-21T10:42:18.3781289Z,1248,,Kings Cross Or Camden Town,2024-06-21T11:03:06Z,2024-06-21T11:03:36Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-1853100398,1,LA19KAX,490015367S,Lady Somerset Road,214,214,GW,outbound,...,,Moorgate Station,2024-06-21T10:42:18.3781289Z,1344,,Camden Town,2024-06-21T11:04:42Z,2024-06-21T11:05:12Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
272,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-1175415875,1,LA19KAX,490010852S2,Parliament Hill Fields,214,214,GK,outbound,...,,Moorgate Station,2024-06-21T10:42:18.3781289Z,1143,,Camden Town,2024-06-21T11:01:21Z,2024-06-21T11:01:51Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
273,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-1996124909,1,LA19KBU,490010852N,Parliament Hill Fields,214,214,GH,inbound,...,,Highgate Village,2024-06-21T10:42:18.3781289Z,514,,Archway Or Highgate Village,2024-06-21T10:50:52Z,2024-06-21T10:51:22Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
274,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-2094866237,1,LA19KBF,490001276S,St Pancras International Station,214,214,S,outbound,...,,Moorgate Station,2024-06-21T10:42:18.3781289Z,1015,,"Farringdon, Holborn Circus Or Angel",2024-06-21T10:59:13Z,2024-06-21T10:59:43Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...


Arrivals based on stop point id (490008660N):


,$type,id,operationType,vehicleId,naptanId,stationName,lineId,lineName,platformName,direction,...,destinationNaptanId,destinationName,timestamp,timeToStation,currentLocation,towards,expectedArrival,timeToLive,modeName,timing
0,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-712048837,1,LA19JZW,490008660N,Lady Somerset Road,214,214,GY,inbound,...,,Highgate Village,2024-06-21T10:41:47.0242847Z,769,,Highgate Village Or Parliament Hill Fields,2024-06-21T10:54:36Z,2024-06-21T10:55:06Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
1,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",418039406,1,LA19KAU,490008660N,Lady Somerset Road,214,214,GY,inbound,...,,Highgate Village,2024-06-21T10:41:47.0242847Z,1624,,Highgate Village Or Parliament Hill Fields,2024-06-21T11:08:51Z,2024-06-21T11:09:21Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
2,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",240807649,1,LA19KBJ,490008660N,Lady Somerset Road,214,214,GY,inbound,...,,Highgate Village,2024-06-21T10:41:47.0242847Z,1108,,Highgate Village Or Parliament Hill Fields,2024-06-21T11:00:15Z,2024-06-21T11:00:45Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
3,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",417973870,1,LA19KBU,490008660N,Lady Somerset Road,214,214,GY,inbound,...,,Highgate Village,2024-06-21T10:41:47.0242847Z,278,,Highgate Village Or Parliament Hill Fields,2024-06-21T10:46:25Z,2024-06-21T10:46:55Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
4,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",1444478742,1,LF67EXD,490008660N,Lady Somerset Road,88,88,GY,inbound,...,,Parliament Hill Fields,2024-06-21T10:41:47.0242847Z,529,,Highgate Village Or Parliament Hill Fields,2024-06-21T10:50:36Z,2024-06-21T10:51:06Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
5,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-595118904,1,YX13BKG,490008660N,Lady Somerset Road,88,88,GY,inbound,...,,Parliament Hill Fields,2024-06-21T10:41:47.0242847Z,1756,,Highgate Village Or Parliament Hill Fields,2024-06-21T11:11:03Z,2024-06-21T11:11:33Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
6,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-1719184039,1,YX18KXU,490008660N,Lady Somerset Road,88,88,GY,inbound,...,,Parliament Hill Fields,2024-06-21T10:41:47.0242847Z,1336,,Highgate Village Or Parliament Hill Fields,2024-06-21T11:04:03Z,2024-06-21T11:04:33Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...


KeyboardInterrupt: 

In [8]:
!pip install requests pandas

import requests
import pandas as pd
import time
from IPython.display import clear_output

# Fetch user inputs
line_id = input("Enter the line id: ")
stop_point_id = input("Enter the stop point id (naptanId): ")
direction = input("Enter the direction (inbound/outbound): ")

# Function to fetch and process data
def fetch_data(line_id, stop_point_id):
    line_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"
    stop_url = f"https://api.tfl.gov.uk/StopPoint/{stop_point_id}/Arrivals"

    try:
        line_response = requests.get(line_url)
        stop_response = requests.get(stop_url)

        line_response.raise_for_status()
        stop_response.raise_for_status()

        line_data = line_response.json()
        stop_data = stop_response.json()

        # Process line data
        line_df = pd.DataFrame(line_data)
        if not line_df.empty:
            line_df = line_df[['lineId', 'vehicleId', 'naptanId', 'stationName', 'direction', 'destinationName', 'timestamp', 'expectedArrival', 'modeName']]

        # Process stop data
        stop_df = pd.DataFrame(stop_data)
        if not stop_df.empty:
            stop_df = stop_df[['lineId', 'vehicleId', 'naptanId', 'stationName', 'direction', 'destinationName', 'timestamp', 'expectedArrival', 'modeName']]

        return line_df, stop_df, None
    except requests.exceptions.RequestException as e:
        return None, None, str(e)

# Main loop to continuously display data
while True:
    clear_output(wait=True)

    line_df, stop_df, error = fetch_data(line_id, stop_point_id)

    if error:
        print("API not responding:", error)
    else:
        print(f"Arrivals based on line id ({line_id}):")
        display(line_df)

        print(f"Arrivals based on stop point id ({stop_point_id}):")
        display(stop_df)

    time.sleep(30)


Arrivals based on line id (214):


,lineId,vehicleId,naptanId,stationName,direction,destinationName,timestamp,expectedArrival,modeName
0,214,LA19KBK,490000036R,Camden Town Station / Camden Street,outbound,Moorgate Station,2024-06-21T10:45:40.9981424Z,2024-06-21T11:06:51Z,bus
1,214,LA19KAU,490012553B,St Pancras Intern'l & King's X Stns,inbound,Highgate Village,2024-06-21T10:45:40.9981424Z,2024-06-21T10:48:48Z,bus
2,214,LA19KBV,490005904E,Penton Rise,outbound,Moorgate Station,2024-06-21T10:45:40.9981424Z,2024-06-21T10:47:07Z,bus
3,214,LA19KAJ,490000169ZA,Old Street Station,inbound,Highgate Village,2024-06-21T10:45:40.9981424Z,2024-06-21T11:09:39Z,bus
4,214,LA19KAO,490000036R,Camden Town Station / Camden Street,outbound,Moorgate Station,2024-06-21T10:45:40.9981424Z,2024-06-21T10:59:41Z,bus
...,...,...,...,...,...,...,...,...,...
263,214,LA19KAX,490015367S,Lady Somerset Road,outbound,Moorgate Station,2024-06-21T10:45:40.9981424Z,2024-06-21T11:04:42Z,bus
264,214,LA19KAX,490010852S2,Parliament Hill Fields,outbound,Moorgate Station,2024-06-21T10:45:40.9981424Z,2024-06-21T11:01:21Z,bus
265,214,LA19KBU,490010852N,Parliament Hill Fields,inbound,Highgate Village,2024-06-21T10:45:40.9981424Z,2024-06-21T10:50:52Z,bus
266,214,LA19KBF,490001276S,St Pancras International Station,outbound,Moorgate Station,2024-06-21T10:45:40.9981424Z,2024-06-21T10:59:13Z,bus


Arrivals based on stop point id (490008660N):


,lineId,vehicleId,naptanId,stationName,direction,destinationName,timestamp,expectedArrival,modeName
0,214,LA19JZW,490008660N,Lady Somerset Road,inbound,Highgate Village,2024-06-21T10:45:09.5135928Z,2024-06-21T10:54:36Z,bus
1,214,LA19KAU,490008660N,Lady Somerset Road,inbound,Highgate Village,2024-06-21T10:45:09.5135928Z,2024-06-21T11:08:51Z,bus
2,214,LA19KBJ,490008660N,Lady Somerset Road,inbound,Highgate Village,2024-06-21T10:45:09.5135928Z,2024-06-21T11:00:15Z,bus
3,214,LA19KBU,490008660N,Lady Somerset Road,inbound,Highgate Village,2024-06-21T10:45:09.5135928Z,2024-06-21T10:47:04Z,bus
4,88,LF67EXD,490008660N,Lady Somerset Road,inbound,Parliament Hill Fields,2024-06-21T10:45:09.5135928Z,2024-06-21T10:53:07Z,bus
5,88,YX13BKG,490008660N,Lady Somerset Road,inbound,Parliament Hill Fields,2024-06-21T10:45:09.5135928Z,2024-06-21T11:11:03Z,bus
6,88,YX18KXU,490008660N,Lady Somerset Road,inbound,Parliament Hill Fields,2024-06-21T10:45:09.5135928Z,2024-06-21T11:04:03Z,bus


KeyboardInterrupt: 

Converting the timestamp and expectedarrival time to GMT+1

In [12]:
!pip install requests pandas
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
import pytz
from IPython.display import clear_output

# Fetch user inputs
line_id = input("Enter the line id: ")
stop_point_id = input("Enter the stop point id (naptanId): ")
direction = input("Enter the direction (inbound/outbound): ")

# Function to fetch and process data
def fetch_data(line_id, stop_point_id):
    line_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"
    stop_url = f"https://api.tfl.gov.uk/StopPoint/{stop_point_id}/Arrivals"

    try:
        line_response = requests.get(line_url)
        stop_response = requests.get(stop_url)

        line_response.raise_for_status()
        stop_response.raise_for_status()

        line_data = line_response.json()
        stop_data = stop_response.json()

        # Process line data
        line_df = pd.DataFrame(line_data)
        if not line_df.empty:
            line_df = line_df[['lineId', 'vehicleId', 'naptanId', 'stationName', 'direction', 'destinationName', 'timestamp', 'expectedArrival', 'modeName']]
            line_df['timestamp'] = line_df['timestamp'].apply(lambda x: process_timestamp(x))
            line_df['expectedArrival'] = line_df['expectedArrival'].apply(lambda x: process_timestamp(x))

        # Process stop data
        stop_df = pd.DataFrame(stop_data)
        if not stop_df.empty:
            stop_df = stop_df[['lineId', 'vehicleId', 'naptanId', 'stationName', 'direction', 'destinationName', 'timestamp', 'expectedArrival', 'modeName']]
            stop_df['timestamp'] = stop_df['timestamp'].apply(lambda x: process_timestamp(x))
            stop_df['expectedArrival'] = stop_df['expectedArrival'].apply(lambda x: process_timestamp(x))

        return line_df, stop_df, None
    except requests.exceptions.RequestException as e:
        return None, None, str(e)

# Function to process timestamp and expectedArrival
def process_timestamp(timestamp):
    if timestamp:
        # Extract time part
        time_part = timestamp.split('T')[1][:5]  # Extract HH:MM

        # Convert to datetime object for arithmetic operations
        dt_obj = datetime.strptime(time_part, '%H:%M')
        dt_obj += timedelta(hours=1)  # Add 1 hour

        # Handle overflow (e.g., when hour becomes 24)
        if dt_obj.hour >= 24:
            dt_obj = dt_obj.replace(hour=dt_obj.hour - 24)

        return dt_obj.strftime('%H:%M')
    else:
        return None

# Main loop to continuously display data
while True:
    clear_output(wait=True)

    line_df, stop_df, error = fetch_data(line_id, stop_point_id)

    if error:
        print("API not responding:", error)
    else:
        print(f"Arrivals based on line id ({line_id}):")
        display(line_df)

        print(f"Arrivals based on stop point id ({stop_point_id}):")
        display(stop_df)

    time.sleep(10)


Arrivals based on line id (214):


,lineId,vehicleId,naptanId,stationName,direction,destinationName,timestamp,expectedArrival,modeName
0,214,LA19KAO,490000169ZA,Old Street Station,inbound,Highgate Village,12:17,12:47,bus
1,214,LA19KAJ,490004714E,Camden High Street,inbound,Highgate Village,12:17,12:38,bus
2,214,LA19KBZ,490000169ZA,Old Street Station,inbound,Highgate Village,12:17,12:32,bus
3,214,LA19KAU,490007082K,Kentish Town Post Office,outbound,Moorgate Station,12:17,12:46,bus
4,214,LA19KAK,490011199R,Prince of Wales Road,inbound,Highgate Village,12:17,12:32,bus
...,...,...,...,...,...,...,...,...,...
235,214,LA19KCE,490010852S2,Parliament Hill Fields,outbound,Moorgate Station,12:17,12:45,bus
236,214,LA19KAK,490012553B,St Pancras Intern'l & King's X Stns,inbound,Highgate Village,12:17,12:17,bus
237,214,LA19KBU,490004937E,Central Street,outbound,Moorgate Station,12:17,12:46,bus
238,214,LA19KBX,490015041Y,Camden Town Station,inbound,Highgate Village,12:17,12:21,bus


Arrivals based on stop point id (490008660N):


,lineId,vehicleId,naptanId,stationName,direction,destinationName,timestamp,expectedArrival,modeName
0,214,LA19KAE,490008660N,Lady Somerset Road,inbound,Highgate Village,12:18,12:43,bus
1,214,LA19KAK,490008660N,Lady Somerset Road,inbound,Highgate Village,12:18,12:37,bus
2,214,LA19KBX,490008660N,Lady Somerset Road,inbound,Highgate Village,12:18,12:31,bus
3,214,LA19KBY,490008660N,Lady Somerset Road,inbound,Highgate Village,12:18,12:22,bus
4,88,YX13BKF,490008660N,Lady Somerset Road,inbound,Parliament Hill Fields,12:18,12:33,bus
5,88,YX13BKG,490008660N,Lady Somerset Road,inbound,Parliament Hill Fields,12:18,12:26,bus
6,88,YX18KXS,490008660N,Lady Somerset Road,inbound,Parliament Hill Fields,12:18,12:37,bus


KeyboardInterrupt: 

Filtering based on direction entered

In [14]:
!pip install requests pandas
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
import pytz
from IPython.display import clear_output

# Fetch user inputs
line_id = input("Enter the line id: ")
stop_point_id = input("Enter the stop point id (naptanId): ")
direction = input("Enter the direction (inbound/outbound): ")

# Function to fetch and process data
def fetch_data(line_id, stop_point_id, direction):
    line_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"
    stop_url = f"https://api.tfl.gov.uk/StopPoint/{stop_point_id}/Arrivals"

    try:
        line_response = requests.get(line_url)
        stop_response = requests.get(stop_url)

        line_response.raise_for_status()
        stop_response.raise_for_status()

        line_data = line_response.json()
        stop_data = stop_response.json()

        # Filter line data based on line_id and direction
        line_df = pd.DataFrame(line_data)
        if not line_df.empty:
            line_df = line_df[(line_df['lineId'] == line_id) & (line_df['direction'] == direction)]
            line_df = line_df[['lineId', 'vehicleId', 'naptanId', 'stationName', 'direction', 'destinationName', 'timestamp', 'expectedArrival', 'modeName']]
            line_df['timestamp'] = line_df['timestamp'].apply(lambda x: process_timestamp(x))
            line_df['expectedArrival'] = line_df['expectedArrival'].apply(lambda x: process_timestamp(x))

        # Filter stop data based on stop_point_id and direction
        stop_df = pd.DataFrame(stop_data)
        if not stop_df.empty:
            stop_df = stop_df[(stop_df['naptanId'] == stop_point_id) & (stop_df['direction'] == direction)]
            stop_df = stop_df[['lineId', 'vehicleId', 'naptanId', 'stationName', 'direction', 'destinationName', 'timestamp', 'expectedArrival', 'modeName']]
            stop_df['timestamp'] = stop_df['timestamp'].apply(lambda x: process_timestamp(x))
            stop_df['expectedArrival'] = stop_df['expectedArrival'].apply(lambda x: process_timestamp(x))

        return line_df, stop_df, None
    except requests.exceptions.RequestException as e:
        return None, None, str(e)

# Function to process timestamp and expectedArrival
def process_timestamp(timestamp):
    if timestamp:
        # Extract time part
        time_part = timestamp.split('T')[1][:5]  # Extract HH:MM

        # Convert to datetime object for arithmetic operations
        dt_obj = datetime.strptime(time_part, '%H:%M')
        dt_obj += timedelta(hours=1)  # Add 1 hour

        # Handle overflow (e.g., when hour becomes 24)
        if dt_obj.hour >= 24:
            dt_obj = dt_obj.replace(hour=dt_obj.hour - 24)

        return dt_obj.strftime('%H:%M')
    else:
        return None

# Main loop to continuously display data
while True:
    clear_output(wait=True)

    line_df, stop_df, error = fetch_data(line_id, stop_point_id, direction)

    if error:
        print("API not responding:", error)
    else:
        print(f"Arrivals based on line id ({line_id}), stop point id ({stop_point_id}), and direction ({direction}):")
        if not line_df.empty:
            print("Line Data:")
            display(line_df)
        else:
            print("No matching data for line id and direction.")

        if not stop_df.empty:
            print("Stop Point Data:")
            display(stop_df)
        else:
            print("No matching data for stop point id and direction.")

    time.sleep(10)


Arrivals based on line id (214), stop point id (490008660N), and direction (inbound):
Line Data:


,lineId,vehicleId,naptanId,stationName,direction,destinationName,timestamp,expectedArrival,modeName
0,214,LA19KAX,490015209C,Finsbury Square,inbound,Highgate Village,12:47,13:01,bus
1,214,LA19KAX,490010109E,Old Street Roundabout,inbound,Highgate Village,12:47,13:05,bus
2,214,LA19KBF,490004714E,Camden High Street,inbound,Highgate Village,12:47,13:11,bus
5,214,LA19KBK,490000169ZA,Old Street Station,inbound,Highgate Village,12:47,12:54,bus
7,214,LA19KBZ,490010928W,Penton Street / Chapel Market,inbound,Highgate Village,12:47,12:48,bus
...,...,...,...,...,...,...,...,...,...
251,214,LA19KBV,490014902N,William Ellis School,inbound,Highgate Village,12:47,13:07,bus
253,214,LA19KAJ,490008660N,Lady Somerset Road,inbound,Highgate Village,12:47,12:55,bus
254,214,LA19KBK,490004937W,Central Street,inbound,Highgate Village,12:47,13:01,bus
257,214,LA19KAJ,490000036L,Kentish Town Road,inbound,Highgate Village,12:47,12:47,bus


Stop Point Data:


,lineId,vehicleId,naptanId,stationName,direction,destinationName,timestamp,expectedArrival,modeName
0,214,LA19KAJ,490008660N,Lady Somerset Road,inbound,Highgate Village,12:47,12:55,bus
1,214,LA19KBO,490008660N,Lady Somerset Road,inbound,Highgate Village,12:47,13:08,bus
2,214,LA19KBV,490008660N,Lady Somerset Road,inbound,Highgate Village,12:47,13:03,bus
3,214,LA19KBZ,490008660N,Lady Somerset Road,inbound,Highgate Village,12:47,13:16,bus
4,88,BD65EVP,490008660N,Lady Somerset Road,inbound,Parliament Hill Fields,12:47,13:04,bus
5,88,YX13BKE,490008660N,Lady Somerset Road,inbound,Parliament Hill Fields,12:47,12:56,bus
6,88,YX18KXY,490008660N,Lady Somerset Road,inbound,Parliament Hill Fields,12:47,13:15,bus


KeyboardInterrupt: 

Changing the time stamp and expected arrival times

In [15]:
!pip install requests pandas

import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from IPython.display import clear_output

# Fetch user inputs
line_id = input("Enter the line id: ")
stop_point_id = input("Enter the stop point id (naptanId): ")
direction = input("Enter the direction (inbound/outbound): ")

# Function to fetch and process data
def fetch_data(line_id, stop_point_id):
    line_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"
    stop_url = f"https://api.tfl.gov.uk/StopPoint/{stop_point_id}/Arrivals"

    try:
        line_response = requests.get(line_url)
        stop_response = requests.get(stop_url)

        line_response.raise_for_status()
        stop_response.raise_for_status()

        line_data = line_response.json()
        stop_data = stop_response.json()

        # Process line data
        line_df = pd.DataFrame(line_data)
        if not line_df.empty:
            line_df = process_data(line_df)

        # Process stop data
        stop_df = pd.DataFrame(stop_data)
        if not stop_df.empty:
            stop_df = process_data(stop_df)

        return line_df, stop_df, None
    except requests.exceptions.RequestException as e:
        return None, None, str(e)

# Function to process timestamp and expectedArrival
def process_data(df):
    if 'timestamp' in df.columns:
        df['timestamp'] = df['timestamp'].apply(lambda x: process_timestamp(x))
    if 'expectedArrival' in df.columns:
        df['expectedArrival'] = df['expectedArrival'].apply(lambda x: process_timestamp(x))
    return df

# Function to process timestamp and expectedArrival
def process_timestamp(timestamp):
    if timestamp:
        # Extract time part
        time_part = timestamp.split('T')[1][:8]  # Extract HH:MM:SS

        # Convert to datetime object for arithmetic operations
        dt_obj = datetime.strptime(time_part, '%H:%M:%S')
        dt_obj += timedelta(hours=1)  # Add 1 hour

        # Handle overflow (e.g., when hour becomes 24)
        if dt_obj.hour >= 24:
            dt_obj = dt_obj.replace(hour=dt_obj.hour - 24)

        return dt_obj.strftime('%H:%M:%S')
    else:
        return None

# Main loop to continuously display data
while True:
    clear_output(wait=True)

    line_df, stop_df, error = fetch_data(line_id, stop_point_id)

    if error:
        print("API not responding:", error)
    else:
        print(f"Arrivals based on line id ({line_id}):")
        display(line_df)

        print(f"Arrivals based on stop point id ({stop_point_id}):")
        display(stop_df)#
    time.sleep(10)


Arrivals based on line id (214):


,$type,id,operationType,vehicleId,naptanId,stationName,lineId,lineName,platformName,direction,...,destinationNaptanId,destinationName,timestamp,timeToStation,currentLocation,towards,expectedArrival,timeToLive,modeName,timing
0,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",869290541,1,LA19KAX,490015209C,Finsbury Square,214,214,C,inbound,...,,Highgate Village,12:52:32,530,,Angel Islington,13:01:22,2024-06-21T12:01:52Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
1,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",175240274,1,LA19KAX,490010109E,Old Street Roundabout,214,214,E,inbound,...,,Highgate Village,12:52:32,751,,Angel Islington,13:05:03,2024-06-21T12:05:33Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
2,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",645020200,1,LA19KAO,490015041Y,Camden Town Station,214,214,Y,inbound,...,,Highgate Village,12:52:32,1688,,Finsbury Park Or Kentish Town,13:20:40,2024-06-21T12:21:10Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
3,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",295086556,1,LA19KBF,490004714E,Camden High Street,214,214,E,inbound,...,,Highgate Village,12:52:32,1116,,Kentish Town,13:11:08,2024-06-21T12:11:38Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
4,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",281498281,1,LA19KBZ,490015041Y,Camden Town Station,214,214,Y,inbound,...,,Highgate Village,12:52:32,939,,Finsbury Park Or Kentish Town,13:08:11,2024-06-21T12:08:41Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-1856884485,1,LA19KAU,490010012E,Moorfields Eye Hospital,214,214,T,outbound,...,,Moorgate Station,12:52:32,1470,,Liverpool Street Or Moorgate,13:17:02,2024-06-21T12:17:32Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
277,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-491598558,1,LA19KBY,490007082K,Kentish Town Post Office,214,214,KC,outbound,...,,Moorgate Station,12:52:32,419,,Camden Town Or Chalk Farm,12:59:31,2024-06-21T12:00:01Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
278,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",320706121,1,LA19KBY,490016573S,Chenies Place,214,214,null,outbound,...,,Moorgate Station,12:52:32,1109,,Angel Or Holborn,13:11:01,2024-06-21T12:11:31Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
279,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",2060017749,1,LA19KAE,490005974S,Gordon House Road,214,214,GN,outbound,...,,Moorgate Station,12:52:32,1507,,Camden Town Or Royal Free Hospital,13:17:39,2024-06-21T12:18:09Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...


Arrivals based on stop point id (490008660N):


,$type,id,operationType,vehicleId,naptanId,stationName,lineId,lineName,platformName,direction,...,destinationNaptanId,destinationName,timestamp,timeToStation,currentLocation,towards,expectedArrival,timeToLive,modeName,timing
0,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",241135329,1,LA19KAJ,490008660N,Lady Somerset Road,214,214,GY,inbound,...,,Highgate Village,12:52:33,233,,Highgate Village Or Parliament Hill Fields,12:56:26,2024-06-21T11:56:56Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
1,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",374721712,1,LA19KBO,490008660N,Lady Somerset Road,214,214,GY,inbound,...,,Highgate Village,12:52:33,962,,Highgate Village Or Parliament Hill Fields,13:08:35,2024-06-21T12:09:05Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
2,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-1153338683,1,LA19KBV,490008660N,Lady Somerset Road,214,214,GY,inbound,...,,Highgate Village,12:52:33,643,,Highgate Village Or Parliament Hill Fields,13:03:16,2024-06-21T12:03:46Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
3,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",551789617,1,LA19KBZ,490008660N,Lady Somerset Road,214,214,GY,inbound,...,,Highgate Village,12:52:33,1418,,Highgate Village Or Parliament Hill Fields,13:16:11,2024-06-21T12:16:41Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
4,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",-1680221132,1,BD65EVP,490008660N,Lady Somerset Road,88,88,GY,inbound,...,,Parliament Hill Fields,12:52:33,715,,Highgate Village Or Parliament Hill Fields,13:04:28,2024-06-21T12:04:58Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
5,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",399911686,1,YX13BKE,490008660N,Lady Somerset Road,88,88,GY,inbound,...,,Parliament Hill Fields,12:52:33,95,,Highgate Village Or Parliament Hill Fields,12:54:08,2024-06-21T11:54:38Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...
6,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",258253517,1,YX18KXY,490008660N,Lady Somerset Road,88,88,GY,inbound,...,,Parliament Hill Fields,12:52:33,1383,,Highgate Village Or Parliament Hill Fields,13:15:36,2024-06-21T12:16:06Z,bus,{'$type': 'Tfl.Api.Presentation.Entities.Predi...


KeyboardInterrupt: 

Limiting the headers

In [16]:
!pip install requests pandas

import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from IPython.display import clear_output

# Fetch user inputs
line_id = input("Enter the line id: ")
stop_point_id = input("Enter the stop point id (naptanId): ")
direction = input("Enter the direction (inbound/outbound): ")

# Function to fetch and process data
def fetch_data(line_id, stop_point_id):
    line_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"
    stop_url = f"https://api.tfl.gov.uk/StopPoint/{stop_point_id}/Arrivals"

    try:
        line_response = requests.get(line_url)
        stop_response = requests.get(stop_url)

        line_response.raise_for_status()
        stop_response.raise_for_status()

        line_data = line_response.json()
        stop_data = stop_response.json()

        # Process line data
        line_df = pd.DataFrame(line_data)
        if not line_df.empty:
            line_df = line_df[['lineId', 'vehicleId', 'naptanId', 'stationName', 'direction', 'destinationName', 'timestamp', 'expectedArrival', 'modeName']]
            line_df['timestamp'] = line_df['timestamp'].apply(process_timestamp)
            line_df['expectedArrival'] = line_df['expectedArrival'].apply(process_timestamp)

        # Process stop data
        stop_df = pd.DataFrame(stop_data)
        if not stop_df.empty:
            stop_df = stop_df[['lineId', 'vehicleId', 'naptanId', 'stationName', 'direction', 'destinationName', 'timestamp', 'expectedArrival', 'modeName']]
            stop_df['timestamp'] = stop_df['timestamp'].apply(process_timestamp)
            stop_df['expectedArrival'] = stop_df['expectedArrival'].apply(process_timestamp)

        return line_df, stop_df, None
    except requests.exceptions.RequestException as e:
        return None, None, str(e)

# Function to process timestamp
def process_timestamp(timestamp):
    if timestamp:
        # Extract time part
        time_part = timestamp.split('T')[1][:8]  # Extract HH:MM:SS

        # Convert to datetime object for arithmetic operations
        dt_obj = datetime.strptime(time_part, '%H:%M:%S')
        dt_obj += timedelta(hours=1)  # Add 1 hour

        # Handle overflow (e.g., when hour becomes 24)
        if dt_obj.hour >= 24:
            dt_obj = dt_obj.replace(hour=dt_obj.hour - 24)

        return dt_obj.strftime('%H:%M:%S')
    else:
        return None

# Main loop to continuously display data
while True:
    clear_output(wait=True)

    line_df, stop_df, error = fetch_data(line_id, stop_point_id)

    if error:
        print("API not responding:", error)
    else:
        print(f"Arrivals based on line id ({line_id}):")
        if not line_df.empty:
            display(line_df)
        else:
            print("No data available.")

        print(f"\nArrivals based on stop point id ({stop_point_id}):")
        if not stop_df.empty:
            display(stop_df)
        else:
            print("No data available.")

    time.sleep(10)


Arrivals based on line id (214):


,lineId,vehicleId,naptanId,stationName,direction,destinationName,timestamp,expectedArrival,modeName
0,214,LA19KAX,490015209C,Finsbury Square,inbound,Highgate Village,12:55:04,13:01:22,bus
1,214,LA19KAX,490010109E,Old Street Roundabout,inbound,Highgate Village,12:55:04,13:05:03,bus
2,214,LA19JZW,490003327H,Angel Islington / City Road,outbound,Moorgate Station,12:55:04,12:59:10,bus
3,214,LA19KBP,490010012E,Moorfields Eye Hospital,outbound,Moorgate Station,12:55:04,12:55:16,bus
4,214,LA19KAO,490015041Y,Camden Town Station,inbound,Highgate Village,12:55:04,13:20:40,bus
...,...,...,...,...,...,...,...,...,...
265,214,LA19KAU,490010012E,Moorfields Eye Hospital,outbound,Moorgate Station,12:55:04,13:17:02,bus
266,214,LA19KBY,490007082K,Kentish Town Post Office,outbound,Moorgate Station,12:55:04,12:59:31,bus
267,214,LA19KBY,490016573S,Chenies Place,outbound,Moorgate Station,12:55:04,13:11:01,bus
268,214,LA19KAE,490005974S,Gordon House Road,outbound,Moorgate Station,12:55:04,13:17:39,bus



Arrivals based on stop point id (490008660N):


,lineId,vehicleId,naptanId,stationName,direction,destinationName,timestamp,expectedArrival,modeName
0,214,LA19KAJ,490008660N,Lady Somerset Road,inbound,Highgate Village,12:55:28,12:56:26,bus
1,214,LA19KBF,490008660N,Lady Somerset Road,inbound,Highgate Village,12:55:28,13:22:50,bus
2,214,LA19KBO,490008660N,Lady Somerset Road,inbound,Highgate Village,12:55:28,13:08:35,bus
3,214,LA19KBV,490008660N,Lady Somerset Road,inbound,Highgate Village,12:55:28,13:04:34,bus
4,214,LA19KBZ,490008660N,Lady Somerset Road,inbound,Highgate Village,12:55:28,13:16:11,bus
5,88,BD65EVP,490008660N,Lady Somerset Road,inbound,Parliament Hill Fields,12:55:28,13:01:55,bus
6,88,YX18KXY,490008660N,Lady Somerset Road,inbound,Parliament Hill Fields,12:55:28,13:15:36,bus


KeyboardInterrupt: 

filter the first api for the given direction, and second api filter for given direction and lineId

In [ ]:
!pip install requests pandas

import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from IPython.display import clear_output

# Fetch user inputs
line_id = input("Enter the line id: ")
stop_point_id = input("Enter the stop point id (naptanId): ")
direction = input("Enter the direction (inbound/outbound): ")

# Function to fetch and process data
def fetch_data(line_id, stop_point_id, direction):
    line_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"
    stop_url = f"https://api.tfl.gov.uk/StopPoint/{stop_point_id}/Arrivals"

    try:
        line_response = requests.get(line_url)
        stop_response = requests.get(stop_url)

        line_response.raise_for_status()
        stop_response.raise_for_status()

        line_data = line_response.json()
        stop_data = stop_response.json()

        # Process line data
        line_df = pd.DataFrame(line_data)
        if not line_df.empty:
            line_df = line_df[line_df['direction'] == direction]  # Filter by direction
            line_df = line_df[['lineId', 'vehicleId', 'naptanId', 'stationName', 'direction', 'destinationName', 'timestamp', 'expectedArrival', 'modeName']]
            line_df['timestamp'] = line_df['timestamp'].apply(process_timestamp)
            line_df['expectedArrival'] = line_df['expectedArrival'].apply(process_timestamp)

        # Process stop data
        stop_df = pd.DataFrame(stop_data)
        if not stop_df.empty:
            stop_df = stop_df[stop_df['direction'] == direction]  # Filter by direction
            stop_df = stop_df[stop_df['lineId'] == line_id]  # Filter by lineId
            stop_df = stop_df[['lineId', 'vehicleId', 'naptanId', 'stationName', 'direction', 'destinationName', 'timestamp', 'expectedArrival', 'modeName']]
            stop_df['timestamp'] = stop_df['timestamp'].apply(process_timestamp)
            stop_df['expectedArrival'] = stop_df['expectedArrival'].apply(process_timestamp)

        return line_df, stop_df, None
    except requests.exceptions.RequestException as e:
        return None, None, str(e)

# Function to process timestamp
def process_timestamp(timestamp):
    if timestamp:
        # Extract time part
        time_part = timestamp.split('T')[1][:8]  # Extract HH:MM:SS

        # Convert to datetime object for arithmetic operations
        dt_obj = datetime.strptime(time_part, '%H:%M:%S')
        dt_obj += timedelta(hours=1)  # Add 1 hour

        # Handle overflow (e.g., when hour becomes 24)
        if dt_obj.hour >= 24:
            dt_obj = dt_obj.replace(hour=dt_obj.hour - 24)

        return dt_obj.strftime('%H:%M:%S')
    else:
        return None

# Main loop to continuously display data
while True:
    clear_output(wait=True)

    line_df, stop_df, error = fetch_data(line_id, stop_point_id, direction)

    if error:
        print("API not responding:", error)
    else:
        print(f"Arrivals based on line id ({line_id}) and direction ({direction}):")
        if not line_df.empty:
            display(line_df)
        else:
            print("No data available.")

        print(f"\nArrivals based on stop point id ({stop_point_id}), line id ({line_id}), and direction ({direction}):")
        if not stop_df.empty:
            display(stop_df)
        else:
            print("No data available.")

    time.sleep(10)


Arrivals based on line id (214) and direction (inbound):


,lineId,vehicleId,naptanId,stationName,direction,destinationName,timestamp,expectedArrival,modeName
0,214,LA19KAO,490014902N,William Ellis School,inbound,Highgate Village,13:11:50,13:32:27,bus
4,214,LA19KBP,490001172X,King's Cross / Caledonian Road,inbound,Highgate Village,13:11:50,13:33:14,bus
5,214,LA19KBK,490004714E,Camden High Street,inbound,Highgate Village,13:11:50,13:25:16,bus
7,214,LA19KBV,490009837N,Merton Lane,inbound,Highgate Village,13:11:50,13:13:43,bus
10,214,LA19KAO,490004894L,Kentish Town Post Office,inbound,Highgate Village,13:11:50,13:26:50,bus
...,...,...,...,...,...,...,...,...,...
285,214,LA19KAO,490007308N,Gordon House Road,inbound,Highgate Village,13:11:50,13:31:08,bus
286,214,LA19KAO,490011199R,Prince of Wales Road,inbound,Highgate Village,13:11:50,13:24:46,bus
287,214,LA19KBO,490005974N,Glenhurst Avenue / Lissenden Gardens,inbound,Highgate Village,13:11:50,13:14:14,bus
288,214,LA19KBF,490011199R,Prince of Wales Road,inbound,Highgate Village,13:11:50,13:18:54,bus



Arrivals based on stop point id (490008660N), line id (214), and direction (inbound):


,lineId,vehicleId,naptanId,stationName,direction,destinationName,timestamp,expectedArrival,modeName
0,214,LA19KAO,490008660N,Lady Somerset Road,inbound,Highgate Village,13:11:53,13:29:53,bus
1,214,LA19KBF,490008660N,Lady Somerset Road,inbound,Highgate Village,13:11:53,13:22:50,bus
2,214,LA19KBK,490008660N,Lady Somerset Road,inbound,Highgate Village,13:11:53,13:37:06,bus
3,214,LA19KBO,490008660N,Lady Somerset Road,inbound,Highgate Village,13:11:53,13:12:06,bus
4,214,LA19KBZ,490008660N,Lady Somerset Road,inbound,Highgate Village,13:11:53,13:19:11,bus
